# Libraries

In [1]:
# Basic libraries
import numpy as np
import pandas as pd

import itertools

# Dataset imports
import json

# For restoring the dataset
from copy import deepcopy

# Text manipulations
import re

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Cosine similarity
from numpy.linalg import norm

c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# NLTK - Natural Language toolkit

In [2]:
!python -m pip install nltk

A library for NLP.
* Usage:
    * nltk.corpus.**stopwords**: stopwords of specific language
    * nltk.tokenize.**RegexpTokenizer**: Tokenize the input sentences
    * nltk.stem.**WordNetLemmatizer**: Lemmatize the word net

In [3]:
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

# Redial Parser
A separated library for parsing the redial dataset

class **RedialParser**
- Restore(): Restore train, test, and movie dataset to initial state
   * return:
        * None
- Movies(train): Get movie list in dataset
   * param:
        * train (bool): Target dataset, (train=True, test=False, all=None)
   * return:
        * dict: {index, MovieName}
- describe(): Describe its datasets
   * return:
        * None
- train: Train data of ReDial.
- test: Test data of ReDial.
- movie: Movie mention counts for ReDial

In [4]:
def load_data(path):
    """
    TODO: initialization function for dataset reads

        :arg
            path (str): Dataset path.
        :return
            tuple: (train, test, df_mention)
    """
    train_data = []
    for line in open(f"{path}/train_data.jsonl", "r"):
        train_data.append(json.loads(line))

    test_data = []
    for line in open(f"{path}/test_data.jsonl", "r"):
        test_data.append(json.loads(line))

    mention_dataframe = pd.read_csv(f"{path}/movies_with_mentions.csv")

    return train_data, test_data, mention_dataframe


class RedialParser:
    def __init__(self, path):
        self.train, self.test, self.movie = load_data(path)

        self._global_movie_list = None  # list of all movie names (global movie name data)
        self._global_msg_list = None  # list of whole lines (global line data)
        self._local_movie_list = None  # list of movie names (local movie name data)
        self._local_msg_list = None  # list of lines (local line data)

        self.dialog_df = None  # Sum of dialogs for each movie indices

        self.__train = deepcopy(self.train)
        self.__test = deepcopy(self.test)
        self.__movie = deepcopy(self.movie)

        self.__model = None


    def Restore(self):
        """
        TODO: Restore train, test, and movie dataset to initial state
        """
        self.train = deepcopy(self.__train)
        self.test = deepcopy(self.__test)
        self.movie = deepcopy(self.__movie)


    def Movies(self, train=True) -> dict:
        """
        TODO: Get movie list in dataset

            :arg
                train (bool): Target dataset, (train=True, test=False, all=None)
            :return
                dict: {index, MovieName}
        """
        if train is None:
            result = self.Movies()
            result.update(self.Movies(False))
            return result

        target = None
        if train is True:
            target = self.train
        elif train is False:
            target = self.test

        result = {}

        if target is not None:
            for elem in target:
                result.update(elem['movieMentions'])

        return result


    def describe(self):
        """
        TODO: Describe its datasets
        """
        len1, len2 = len(self.train), len(self.test)
        n1, n2 = 0, 0
        m1, m2 = 0, 0

        for e in self.train:
            n1 += len(e['movieMentions'])
            m1 += len(e['messages'])
        for e in self.test:
            n2 += len(e['movieMentions'])
            m2 += len(e['messages'])

        print('Brief information:\n'
              f'Length of train data: {len1}\n'
              f'Length of test data: {len2}\n\n'
              'Data information:\n'
              f'Key parameters: {list(self.train[0].keys())}\n'
              f'Key parameters in Questions: {list(list(self.train[0]["respondentQuestions"].values())[0].keys())}\n'
              f'Key parameters in messages: {list(self.train[0]["messages"][0].keys())}\n\n'
              'Context information:\n'
              f'Total mentioned movie number (train): {n1}\n'
              f'Total mentioned movie number in unique (train): {len(self.Movies())}\n'
              f'Total message number (train): {m1}\n'
              f'Total mentioned movie number (test): {n2}\n'
              f'Total mentioned movie number in unique (test): {len(self.Movies(False))}\n'
              f'Total message number (test): {m2}\n'
              f'Average mentioned movie numbers per conversation (train): {n1 / len1}\n'
              f'Average message numbers per conversation (train): {m1 / len1}\n'
              f'Average mentioned movie numbers per conversation (test): {n2 / len2}\n'
              f'Average message numbers per conversation (test): {m2 / len2}\n\n'
              , end='')
    

    def preprocessing(self):
        """
        TODO: Regroup train dataset into purposed structure and clean up data
        """
        compile = re.compile("\W+")  # Format
        
        ran = range(len(self.train))

        # initialize list
        self._global_movie_list = []
        self._global_msg_list = []
        self._local_movie_list = [[] for _ in ran]
        self._local_msg_list = [[] for _ in ran]

        for i, data in enumerate(self.train):
            for msg in data['messages']:  # append line to the lists
                self._local_msg_list[i].append(msg['text'])
                self._global_msg_list.append(msg['text'])

            # Extract movie indices
            for idx, line in enumerate(self._local_msg_list[i]):
                numbers = re.findall(r'@\d+', line)  # find number keywords (ex: @12345)
                for number in numbers:
                    self._local_movie_list[i].append(number[1:])
                    self._global_movie_list.append(number[1:])

                    # Remove index string
                    pos = line.index(number)
                    line = self._local_msg_list[i][idx] = line[0: pos] + line[pos + len(number): len(line)]

                # Post: clear meaningless words
                a = compile.sub(" ", line)  # Clear special character
                self._local_msg_list[i][idx] = a.lower()  # lower character

        # Construct dialog dataframe
        self.dialog_df = pd.DataFrame(columns=["movieid", "dialog"])

        for lines, movies in zip(self._local_msg_list, self._local_movie_list):
            dig = ''
            for line in lines:  # concatenate all sentences in related message dialog
                dig += ' ' + str(line)
            
            for mv in movies:
                newrow = pd.DataFrame({'movieid': [mv], 'dialog': [dig]}, columns=self.dialog_df.columns)
                self.dialog_df = pd.concat([self.dialog_df, newrow], ignore_index=True)
        
        # Fill NaN with empty sentence
        self.dialog_df['dialog'].fillna('', inplace=True)
    

    def get_frequency_matrix(self, tags):
        """
        TODO: compute the frequency of tag words to obtain the TF-IDFs matrix

            :arg
                tags (list): list of key words.
            :return
                pandas.DataFrame: frequency matrix of tag words.
        """
        stop_word_eng = set(stopwords.words('english'))
        ran = range(len(self.train))

        msg_list = deepcopy(self._local_msg_list)

        for i in ran:
            msg_list[i] = [j for j in msg_list[i] if j not in stop_word_eng]  # Clear stopwords

        # Lemmatizer class
        lemmatizer = WordNetLemmatizer()
        token = RegexpTokenizer('[\w]+')

        # mv_tags = ['comedy','scary','love','animation','artistic','war','sci','blood','hero','romantic','action']
        x = pd.DataFrame(columns=['id'] + tags)

        for idx, msg in enumerate(msg_list):
            result_pre_lem = [token.tokenize(j) for j in msg]
            middle_pre_lem = [r for j in result_pre_lem for r in j]
            final_lem = [lemmatizer.lemmatize(j) for j in middle_pre_lem if not j in stop_word_eng]  # Remove stopword

            # Lemmatization
            english = pd.Series(final_lem)
            for word in english:
                if word in tags:
                    for movie in self._local_movie_list[idx]:
                        if x[x['id'] == movie].empty:
                            new_row = pd.DataFrame({'id': [movie]}, columns=x.columns)
                            x = pd.concat([x, new_row], ignore_index=True)
                            x.fillna(0, inplace=True)
                        x.loc[x['id'] == movie, word] += 1

        return x
    
    def get_tfidf_matrix(self, **tfidf_keys):
        """
        TODO: Compute TF-IDFs matrix

            :arg
                tfidf_keys(keyword dict): TfidfVectorizer parameters
            :return
                numpy.ndrarry: TF-IDFs matrix
                numpy.ndarray: feature name of TF-IDFs (word)
        """
        # Vectorizer class
        tfidf = TfidfVectorizer(**tfidf_keys)  # Ignore English Stopwords

        # Obtain matrix
        tfidf_df = tfidf.fit_transform(self.dialog_df['dialog'])

        return tfidf_df.toarray(), tfidf.get_feature_names_out()


    def __max_sum_sim(self, doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
        """
        TODO: Minimize redundancy and maximize diversity - Get {top_n} keyword(s) using Cosine Similarity

            :arg
                doc_embedding(numpy.ndarray): Document embedding (Obtained from dialog)
                candidate_embeddings(numpy.ndarray): Candidates embedding (Obtained from CountVectorizer)
                words(numpy.ndarray): Candidates
                top_n(int): Maximum size of high similarity indices
                nr_candidates(int): Nearby Candidates, the word list has maximum size is this.
            :return
                list: list of high similarity keywords with length = {top_n}
        """
        # Compute similarity of document and candidates
        distances = cosine_similarity(doc_embedding, candidate_embeddings)

        # Compute similarity of candidates
        distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

        # Explict {top_n} words
        words_idx = list(distances.argsort()[0][-nr_candidates:])
        words_vals = [words[index] for index in words_idx]
        distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

        # Compute the combination of keywords having minimum redundancy
        min_sim = np.inf
        candidate = None
        for combination in itertools.combinations(range(len(words_idx)), top_n):
            sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
            if sim < min_sim:
                candidate = combination
                min_sim = sim

        return [words_vals[idx] for idx in candidate]
    
    
    def __mmr(self, doc_embedding, candidate_embeddings, words, top_n, diversity):
        """
        TODO: Minimize redundancy and maximize diversity - Get {top_n} keyword(s) using MMR

            :arg
                doc_embedding(numpy.ndarray): Document embedding (Obtained from dialog)
                candidate_embeddings(numpy.ndarray): Candidates embedding (Obtained from CountVectorizer)
                words(numpy.ndarray): Candidates
                top_n(int): Maximum size of high similarity indices
                diversity(float): Diversity of candidates, higher diversity has larger candidates num.
            :return
                list: list of high similarity keywords with length = {top_n}
        """
        # Word - document similarity
        word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

        # Candidates - similarity
        word_similarity = cosine_similarity(candidate_embeddings, candidate_embeddings)

        # Extract the simliarest word's index
        keywords_idx = [np.argmax(word_doc_similarity)]

        # Index list excluding the similarest index
        candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

        # Repeat this for remaining counts (top_n - 1)
        for _ in range(top_n - 1):
            candidate_similarities = word_doc_similarity[candidates_idx, :]
            print(word_similarity.shape, candidates_idx, keywords_idx)
            target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

            # Compute the MMR
            mmr = (1 - diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
            mmr_idx = candidates_idx[np.argmax(mmr)]

            # Update indices
            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)

        return [words[idx] for idx in keywords_idx]
    

    def BERT(self, bert_model, top_n):
        """
            TODO: Do *BERT* model using CountVectorizer

            :arg
                bert_model(str): Sentence Transofmer name, see more information in https://huggingface.co/models?library=sentence-transformers
                top_n(int): The number of high similarity words from candidates
            :return
                pandas.DataFrame: Movie index and {top_n} words with high similarity using cosine_similarity and MMR.
        """
        num = len(self.dialog_df['movieid'])

        n_gram_range = (3, 3)
        
        df = pd.DataFrame(columns=['movieid', 'Words(cosine_similarity)', 'Words(MMR)'])

        for i in range(num):
            count = CountVectorizer(ngram_range=n_gram_range, stop_words='english').fit(self.dialog_df.iloc[i, :].values)
            candidates = count.get_feature_names_out()

            model = SentenceTransformer(bert_model)

            doc_embedding = model.encode(self.dialog_df.iloc[i, :].values)
            candidate_embeddings = model.encode(candidates)

            (csim, mmr) = (self.__max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n, 20),
                self.__mmr(doc_embedding, candidate_embeddings, candidates, top_n, 0.7))

            newrow = pd.DataFrame({
                'movieid': self.dialog_df.iloc[i, 0].values,
                'Words(cosine_similarity)': [csim],
                'Words(MMR)': [mmr]},
                columns=df.columns)
            df = pd.concat([df, newrow], ignore_index=True)
        
        return df
    

    def similarity(X, Y):
        """
        TODO: Compute the cosine simliarity between X and Y. For avoiding the DivByZero, the denominator has 1e-7 minimum value.

            :arg
                X (numpy.ndarray): X data array
                Y (numpy.ndarray): Y data array
            :return
                float
        """
        return np.dot(X, Y) / ((norm(X) * norm(Y)) + 1e-7)


# Initialize
Import dataset, describe it briefly.

In [5]:
parser = RedialParser('../dataset')
parser.describe()  # Describe read dataset

# Size of train data
num = len(parser.train)
print(f'length of train dataset: {num}')

Brief information:
Length of train data: 10006
Length of test data: 1342

Data information:
Key parameters: ['movieMentions', 'respondentQuestions', 'messages', 'conversationId', 'respondentWorkerId', 'initiatorWorkerId', 'initiatorQuestions']
Key parameters in Questions: ['suggested', 'seen', 'liked']
Key parameters in messages: ['timeOffset', 'text', 'senderWorkerId', 'messageId']

Context information:
Total mentioned movie number (train): 52918
Total mentioned movie number in unique (train): 6223
Total message number (train): 182150
Total mentioned movie number (test): 7154
Total mentioned movie number in unique (test): 2007
Total message number (test): 23952
Average mentioned movie numbers per conversation (train): 5.288626823905656
Average message numbers per conversation (train): 18.20407755346792
Average mentioned movie numbers per conversation (test): 5.330849478390462
Average message numbers per conversation (test): 17.847988077496275

length of train dataset: 10006


# Preprocessing
Clear the special character and extract the text and movie indices
- example: "I like animations like @84779 and @191602" → [i like animations like  and ], [84779, 191602]


Specific:
* Transform dataset structure.
    * Original: [movieMentions, {messages}, conversationId, ...]
    * Transformed: [movie_indices], [message_contexts], [[1st_movie_index], [2nd_...], ...], [[1st_message_context], [2nd_...], ...]
    * Dialog Dataframe (*self.dialog_df*): {'movie_id': '1st message' + '2nd message' + ...} - Used in generation of **TF-IDF** matrix
* Recognize movie indices
    * **@** recognition: use re library's *findall(@\d+)* function, it only detects '@' + index strings.
* Clean up meaningless values
    * Special characters: use re library's format *\w+*, it only receives widechar characters.
    * Movie index: remove context of them by using text slicing.

In [6]:
parser.preprocessing()
parser.dialog_df

,movieid,dialog
0,84779,hi there how are you i m looking for movie re...
1,191602,hi there how are you i m looking for movie re...
2,122159,hi there how are you i m looking for movie re...
3,165710,hi there how are you i m looking for movie re...
4,151313,hi there how are you i m looking for movie re...
...,...,...
64456,204974,what type of movies do you like hi i m looki...
64457,85036,hello hi how can i help you so some of the m...
64458,170277,hello hi how can i help you so some of the m...
64459,149938,hello hi how can i help you so some of the m...


# Tokenization
* 1. Extract words and their counts related to the movies. (Did not used, only for eye inspection.)

In [7]:
# Tag words words related with movie genres
mv_tags = ['comedy','scary','love','animation','artistic','war','sci','blood','hero','romantic','action']
frequency = parser.get_frequency_matrix(mv_tags)
frequency

,id,comedy,scary,love,animation,artistic,war,sci,blood,hero,romantic,action
0,84779,2,0,0,1,1,0,1,0,0,0,1
1,191602,2,0,0,1,1,0,1,0,0,0,1
2,122159,2,0,0,1,1,0,1,0,0,0,1
3,165710,37,1,95,13,1,0,4,0,1,0,13
4,151313,3,0,2,3,1,0,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
5096,205981,0,0,0,1,0,0,0,0,0,0,0
5097,106113,0,0,0,0,0,0,3,0,0,0,1
5098,96852,0,0,0,0,0,0,0,0,0,0,1
5099,112404,0,0,0,0,0,0,0,0,0,0,1


* 2. Normal TF-IDF

In [8]:
tfidf_mat, tfidf_columns = parser.get_tfidf_matrix(stop_words='english', min_df=0.2)

# Construct dataset with id + word vectors
cdata = np.concatenate((parser.dialog_df['movieid'].to_numpy().reshape(len(parser.dialog_df['dialog']), 1), tfidf_mat), axis=1)
df_mv_tfidf = pd.DataFrame(cdata, columns=['id'] + tfidf_columns.tolist())
df_mv_tfidf

,id,bye,check,comedy,day,did,enjoy,funny,good,great,...,saw,seen,suggestions,sure,thank,thanks,think,ve,watch,yes
0,84779,0.155447,0.0,0.232591,0.0,0.0,0.226592,0.234526,0.20677,0.0,...,0.0,0.0,0.0,0.0,0.0,0.147461,0.0,0.0,0.0,0.0
1,191602,0.155447,0.0,0.232591,0.0,0.0,0.226592,0.234526,0.20677,0.0,...,0.0,0.0,0.0,0.0,0.0,0.147461,0.0,0.0,0.0,0.0
2,122159,0.155447,0.0,0.232591,0.0,0.0,0.226592,0.234526,0.20677,0.0,...,0.0,0.0,0.0,0.0,0.0,0.147461,0.0,0.0,0.0,0.0
3,165710,0.155447,0.0,0.232591,0.0,0.0,0.226592,0.234526,0.20677,0.0,...,0.0,0.0,0.0,0.0,0.0,0.147461,0.0,0.0,0.0,0.0
4,151313,0.155447,0.0,0.232591,0.0,0.0,0.226592,0.234526,0.20677,0.0,...,0.0,0.0,0.0,0.0,0.0,0.147461,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64456,204974,0.0,0.0,0.0,0.0,0.173777,0.0,0.0,0.242174,0.0,...,0.182008,0.085112,0.297298,0.0,0.0,0.11514,0.0,0.0,0.139028,0.0
64457,85036,0.0,0.0,0.0,0.4115,0.0,0.248506,0.0,0.170075,0.070963,...,0.127822,0.059773,0.0,0.253538,0.290231,0.0,0.304353,0.0,0.292913,0.178612
64458,170277,0.0,0.0,0.0,0.4115,0.0,0.248506,0.0,0.170075,0.070963,...,0.127822,0.059773,0.0,0.253538,0.290231,0.0,0.304353,0.0,0.292913,0.178612
64459,149938,0.0,0.0,0.0,0.4115,0.0,0.248506,0.0,0.170075,0.070963,...,0.127822,0.059773,0.0,0.253538,0.290231,0.0,0.304353,0.0,0.292913,0.178612


* 3. BERT

In [9]:
bert_df = parser.BERT('distilbert-base-nli-mean-tokens', 5)
bert_df

IndexError: index 68 is out of bounds for axis 1 with size 57

# Similarity Metrics
* Cosine similarity

Recommendation function
* param:
    * data: array, vector space of texts.
    * mv: target movie's index
    * length: maximum length of recommendation
        * default: 5
    * simf: consine similarity function
        * default: dot(X, y) / (normalize(X) * normalize(Y) + 1e-7)

In [ ]:
# Note: the consine similarity function's denominator has 1e-7 minimum value to avoid the divbyzero.
def recommend(matrix, index, length=5, simf):
    sim = []

    if df.loc[df['movieid'] == mv].empty:
        return sim
    
    idx = df[df['movieid'] == mv].index.values[0]

    for idx, data in enumerate(matrix):
        if idx != i:
            sim.append((simf(data[i], data[idx]), df.loc[i]['movieid']))
    
    sim.sort()
    sim.reverse()
    return sim[:length]